In [2]:
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
%matplotlib inline
import argparse
import sys, getopt

In [3]:
def detect(img, cascade):
    rects = cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags = cv2.CASCADE_SCALE_IMAGE)
    if len(rects) == 0:
        return []
    rects[:,2:] += rects[:,:2]
    return rects


In [4]:
def draw_rects(img, rects, color):
    for x1, y1, x2, y2 in rects:
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

In [5]:
#parser = argparse.ArgumentParser()    
#parser.add_argument("input", help="the file name")    
#args = parser.parse_args()

video_src = 'VIDEO.MPG'#args.input
#args = dict(args)
#cascade_fn = 'C:\\opencv\\build\\etc\\haarcascades\\haarcascade_frontalface_default.xml' #args.get('--cascade', "../../data/haarcascades/haarcascade_frontalface_alt.xml")
#nested_fn  = 'C:\\opencv\\build\\etc\\haarcascades\\haarcascade_eye.xml' #args.get('--nested-cascade', "../../data/haarcascades/haarcascade_eye.xml")

cascade = cv2.CascadeClassifier('C:\\Users\\Home\\Anaconda3\\pkgs\\opencv-3.3.1-py36h20b85fd_1\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml')
nested = cv2.CascadeClassifier('C:\\Users\\Home\\Anaconda3\\pkgs\\opencv-3.3.1-py36h20b85fd_1\\Library\\etc\\haarcascades\\haarcascade_eye.xml')



In [6]:
cam = cv2.VideoCapture(video_src)
filter_img = cv2.imread('hearts.png')
height, width, channels = filter_img.shape

print(height, width)

338 611


In [23]:

while True:
    ret, img = cam.read()
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)

    #t = clock()
    rects = detect(gray, cascade)
    vis = img.copy()
    
    if(rects != []):
        #print(rects[0:1])
        x1,y1,x2,y2 = rects[0]
        new_width = int(x2 - x1)
        new_height = int((y2-y1)/4)
       

        filter_scaled = cv2.resize(filter_img,(new_width, new_height))
      
        
        #print(new_width,new_height)
        #print( x1,y1,x2,y2)
        
        xt = x1
        yt = y1
        for w in range(new_width-1):
            
            for h in range(new_height-1):
                if(np.average(filter_scaled[h,w]) > 0):
                    vis[yt,xt] = filter_scaled[h,w]
                yt += 1
            yt = y1
            xt += 1
    
    #draw_rects(vis, rects, (0, 255, 0))
    #for x1, y1, x2, y2 in rects:
    #    roi = gray[y1:y2, x1:x2]
    #    vis_roi = vis[y1:y2, x1:x2]
    #    subrects = detect(roi.copy(), nested)
    #    draw_rects(vis_roi, subrects, (255, 0, 0))
    #dt = clock() - t

    #draw_str(vis, (20, 20), 'time: %.1f ms' % (dt*1000))
    cv2.imshow('facedetect', vis)

    if 0xFF & cv2.waitKey(5) == 27:
        break
cv2.destroyAllWindows()

C:\Users\Home\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
  if sys.path[0] == '':


error: C:\ci\opencv_1512688052760\work\modules\imgproc\src\color.cpp:11048: error: (-215) scn == 3 || scn == 4 in function cv::cvtColor
